In [366]:
## Calculate the summary statistics for the single-cell datasets similar as in D1
## Compared to D1: only calculate for a subset of genes (e.g. random sample, only co-eQTL genes etc + include estimation of negative binomial distribution parameters
## Used as input for simulation

# Libraries

In [367]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'"


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"


# Parameters

In [368]:
### Path to single-cell datasets (oneK1K, Wijst, UMCG2, UMCG3, multiome; stored as .Rds data files)

data_path_sc = "../data/current/wg3/"

In [370]:
### Path to save calculated summary statistics to
output_path_sc = "../data/current/summary_stats/"


In [ ]:
### Select cell-type and gene list to calculate the summary stats for

In [495]:
cell_type = 'NK'

In [496]:
gene_list_path =  "/ictstr01/groups/epigenereg01/workspace/projects/grn_dev_groningen/coeqtl_mapping/co_qtls_decision_tree/shared_results/5DS_Meta_Analysis_Sign_coeQTLs133p_val_mt_eGene_.csv"
#gene_list_path= paste0('../data/current/coeqtl_mapping/co_qtls_decision_tree/analysis_oneK1K/', cell_type , '/')
# '../data/current/coeqtl_mapping/co_qtls_sceqtlgen/oneK1K/CD4_T/'   # path to significant genes from co-eqtl mapping

# Functions

In [497]:
source('MS1_Functions.r')

# Data

## Load different sc dataset and get some meta-information

In [ ]:
### Select datasets for which summary statistics should be calculated

In [499]:
folders = c('wg3_wijst2018'#, 
            #'wg3_Trynka', 
           # 'wg3_oneK1k'#,
           # 'wg3_NC2020',
            #'wg3_Nawijn',
            #'wg3_Li',
           # 'wg3_Ye',
           #  'wg3_okada',
           #   'wg3_sawcer',
           # 'wg3_Franke_split_v3',
          #  'wg3_Franke_split_v2',
          #  'wg3_blokland2020'
           )  ## specify folders that contain datasets that should be included

In [500]:
data = list()

In [501]:
overview_data = data.frame()

In [502]:
## Get all available datasets overview
for(i in folders){

    dataset_overview = data.frame( datasets = list.files(paste0(data_path_sc, i, '/input/L1')))
    if(nrow(dataset_overview) >= 1){
        dataset_overview$name = i
        dataset_overview = dataset_overview[!is.na(str_extract(dataset_overview$datasets, '.Rds')),]
        overview_data = rbind(overview_data, dataset_overview)
      }
    }


In [503]:
dataset_overview

,datasets,name
,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_wijst2018
2,CD4_T.Qced.Normalized.SCs.Rds,wg3_wijst2018
3,CD8_T.Qced.Normalized.SCs.Rds,wg3_wijst2018
4,Mono.Qced.Normalized.SCs.Rds,wg3_wijst2018
5,NK.Qced.Normalized.SCs.Rds,wg3_wijst2018


In [504]:
head(overview_data,20)

,datasets,name
,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_wijst2018
2,CD4_T.Qced.Normalized.SCs.Rds,wg3_wijst2018
3,CD8_T.Qced.Normalized.SCs.Rds,wg3_wijst2018
4,Mono.Qced.Normalized.SCs.Rds,wg3_wijst2018
5,NK.Qced.Normalized.SCs.Rds,wg3_wijst2018


In [505]:
overview_data$datasets_match = paste0(overview_data$name, overview_data$datasets)

In [506]:
overview_data[1,]

,datasets,name,datasets_match
,<chr>,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018B.Qced.Normalized.SCs.Rds


In [507]:
length(unique(overview_data$name))

[1] 1

In [508]:
nrow(overview_data)

[1] 5

In [509]:
## Function to load dataset
get_dataset = function(path){
    data = readRDS(path)
    }

In [510]:
## Get already generated summaries

In [511]:
available_summaries = data.frame()

In [512]:
available_summaries= data.frame( datasets = list.files(paste0(output_path_sc)))
available_summaries = data.frame(datasets = available_summaries[!is.na(str_extract(available_summaries$datasets, 'D1_Sample_cell_statistic')),])
    

In [513]:
available_summaries$datasets = str_replace(available_summaries$datasets, 'D1_Sample_cell_statistic', '')
available_summaries$datasets = str_replace(available_summaries$datasets, '.csv', '')

In [514]:
nrow(available_summaries)

[1] 177

In [515]:
head(available_summaries,2)

,datasets
,<chr>
1,wg3_blokland2020B.Qced.Normalized.SCs.Rds
2,wg3_blokland2020CD4_T.Qced.Normalized.SCs.Rds


In [516]:
nrow(overview_data[overview_data$datasets_match %in% available_summaries$datasets,]) # already existing summaries

[1] 5

In [517]:
## Remove already existing sommaries from overview data to avoid rerunning

In [518]:
calculate_new = TRUE

In [519]:
if(calculate_new == FALSE){
    overview_data = overview_data[!overview_data$datasets_match %in% available_summaries$datasets,]
    }
    

In [520]:
nrow(overview_data)

[1] 5

In [521]:
##Remove tmp Filtered objects

In [522]:
overview_data = overview_data[is.na(str_extract(overview_data$datasets, 'tmpFiltered')),]

In [523]:
#overview_data

In [524]:
head(overview_data,2)

,datasets,name,datasets_match
,<chr>,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018B.Qced.Normalized.SCs.Rds
2,CD4_T.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018CD4_T.Qced.Normalized.SCs.Rds


In [525]:
overview_data

,datasets,name,datasets_match
,<chr>,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018B.Qced.Normalized.SCs.Rds
2,CD4_T.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018CD4_T.Qced.Normalized.SCs.Rds
3,CD8_T.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018CD8_T.Qced.Normalized.SCs.Rds
4,Mono.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018Mono.Qced.Normalized.SCs.Rds
5,NK.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018NK.Qced.Normalized.SCs.Rds


In [526]:
### Filter on cell-type which was selected

In [527]:
overview_data = overview_data[!is.na(str_extract(overview_data$datasets, cell_type)),]

In [528]:
overview_data

,datasets,name,datasets_match
,<chr>,<chr>,<chr>
5,NK.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018NK.Qced.Normalized.SCs.Rds


## Load the list of genes for which to calculate

In [529]:
gene_selection = 'random' # options: random, coeqtl --> random 2000 random genes will be chosen; coeqtl: significant co-eqtl genes are chosen

In [530]:
### Use significant co-eQTLs

In [531]:
gene_list_path

[1] "/ictstr01/groups/epigenereg01/workspace/projects/grn_dev_groningen/coeqtl_mapping/co_qtls_decision_tree/shared_results/5DS_Meta_Analysis_Sign_coeQTLs133p_val_mt_eGene_.csv"

In [532]:
coeqtls = fread(paste0(gene_list_path), header = TRUE)

# 'F3_decision_tree_input.csv'

In [533]:
## Filter which ones to consider as significant

In [535]:
nrow(coeqtls)

[1] 91562

In [536]:
length(unique(coeqtls$triplet))

[1] 64198

In [537]:
coeqtls = data.frame(coeqtls)

In [538]:
### Filter on the cell-type
coeqtls = coeqtls[coeqtls$cell_type == cell_type,]

In [539]:
head(coeqtls,2)

,V1,eGene,coeGene,snp_id,triplet,feature_id,cell_type,dataset,n_samples,p_val_mt,⋯,z_score.oneK1K.CD4_T,z_score.oneK1K.CD8_T,z_score.oneK1K.Mono,z_score.oneK1K.NK,z_score.wijst.B,z_score.wijst.CD4_T,z_score.wijst.CD8_T,z_score.wijst.Mono,z_score.wijst.NK,remap_overlap_tf_snp
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
373,373,AK5,CASK,1:77443300:A:G,1:77443300:A:G;AK5_CASK,AK5_CASK,NK,meta_5ds,499,2.170199e-07,⋯,NA,NA,NA,7.877279,NA,NA,NA,NA,NA,NA
374,374,AK5,CD247,1:77443300:A:G,1:77443300:A:G;AK5_CD247,AK5_CD247,NK,meta_5ds,630,2.838419e-07,⋯,NA,NA,NA,-7.947331,NA,NA,NA,NA,NA,NA


In [540]:
## Alternative randomly select 2000 genes

In [541]:
path = paste0(data_path_sc, unique(overview_data$name), '/input/L1/', unique(overview_data$datasets))
data = get_dataset(path)   

In [542]:
if(gene_selection == 'coeqtl'){
    genes = unique(c(coeqtls$eGene, coeqtls$coeGene))
    }
if(gene_selection == 'random'){
    set.seed(1)
    genes = sample(unique(rownames(data)),5000)
    }

In [543]:
length(genes)

[1] 5000

In [544]:
length(unique(coeqtls$eGene))

[1] 49

In [545]:
length(unique(coeqtls$coeGene))

[1] 457

In [546]:
#sum(unique(coeqtls$gene1) %in% unique(coeqtls$gene2))

# Get summary statistics of input data and save

In [547]:
folders

[1] "wg3_wijst2018"

In [548]:
overview_data$data = overview_data$datasets

In [549]:
sort(unique(overview_data$data))

[1] "NK.Qced.Normalized.SCs.Rds"

In [550]:
choose_assay = 'RNA' # alternative: assay = 'data' --> use 'data' for normalized values; 'RNA' for raw counts

In [ ]:
### Generate empty dataframes to store calculated summaries in

In [551]:
sample_summary = data.frame()

In [552]:
sample_gene_statistic = data.frame()

In [553]:
sample_cell_statistic = data.frame()

In [554]:
overview_data$data[1]

[1] "NK.Qced.Normalized.SCs.Rds"

In [555]:
overview_data

,datasets,name,datasets_match,data
,<chr>,<chr>,<chr>,<chr>
5,NK.Qced.Normalized.SCs.Rds,wg3_wijst2018,wg3_wijst2018NK.Qced.Normalized.SCs.Rds,NK.Qced.Normalized.SCs.Rds


In [ ]:
#### Calculate summary statistics

In [556]:
for(i in 1:nrow(overview_data)){
    path = paste0(data_path_sc, overview_data$name[i], '/input/L1/', overview_data$data[i])
    data = get_dataset(path)   
    data = data[genes]  # subsample on the selected genes
    #print(data)
    #print(unique(data$Assignment))
    
    ## Get overview on amount samples: sample_summary
    sample_data = data[[]] %>% group_by(Assignment) %>% summarise(amount_cells = n())
    sample_data$dataset = overview_data$name[i]
    sample_data$cell_type = overview_data$data[i]
    sample_summary = rbind(sample_summary, sample_data)  # TBD outside of loop!


    for(k in unique(data$Assignment)) {     
            
        ## Get summary for each gene and sample: sample_gene_statistic
        sample_data = data[,data$Assignment == k ]
        #sample_data_trans = GetAssayData(sample_data ,assay = 'data')
        sample_data = GetAssayData(sample_data ,assay =choose_assay)  # alternative: assay = 'data' --> returns transformed values

            ### Filter genes with at leat 10 non-zero counts
            sample_data = sample_data[rowSums(sample_data >= 1) >= 10,]

            ### convert format
            sample_data = as.matrix(sample_data)
            #sample_data_trans = as.matrix(sample_data_trans)

            data_long = melt(sample_data)
            #data_long_trans = melt(sample_data_trans)

            colnames(data_long) = c('gene', 'cell', 'counts')
            #colnames(data_long_trans) = c('gene', 'cell', 'trans_counts')

            ### get summaries
            summary_data = data_long %>% group_by(gene) %>% summarise(
                amount_cells = n(),
                amount_zero_counts = sum(counts == 0),
                mean_counts = mean(counts, na.rm = TRUE),
                max_counts = max(counts, na.rm = TRUE),
                min_counts = min(counts, na.rm = TRUE),
                variance_counts = var(counts, na.rm = TRUE),
                bin_size = as.numeric(try(fitdistr(counts, 'negative binomial')$estimate[1])), 
                bin_mu= as.numeric(try( fitdistr(counts, 'negative binomial')$estimate[2]))

            )

            summary_data$percentage_zero = summary_data$amount_zero_counts / summary_data$amount_cells
            summary_data$amount_non_zero_counts = summary_data$amount_cells - summary_data$amount_zero_counts
            summary_data$percentage_zero =  summary_data$percentage_zero*100
            ## bin different columns
            summary_data =  summary_data %>% mutate(amount_cells_bin = cut(amount_cells, breaks=c(0,25, 50, 100,250, 500, 750, 1500)))
            summary_data =  summary_data %>% mutate(percentage_zero_bin = cut(percentage_zero, breaks=c(-0.001,5, 10, 20,40, 60, 80,90, 95, 100)))


            summary_data$sample = k
            summary_data$cell_type = overview_data$data[i]
            summary_data$dataset = overview_data$name[i]

            sample_gene_statistic = rbind(sample_gene_statistic, summary_data)
        #
        
        # Get summaries for each sample and each cell: sample_cell_statistic
        sample_data = data[,data$Assignment == k ]
        sample_data = GetAssayData(sample_data ,assay =choose_assay)  # alternative: assay = 'data' --> returns transformed values

            ### convert format
            sample_data = as.matrix(sample_data)

            data_long = melt(sample_data)

            colnames(data_long) = c('gene', 'cell', 'counts')

            ### get summaries
            summary_data = data_long %>% group_by(cell) %>% summarise(
                amount_counts = sum(counts),
                amount_genes = n(),
                amount_zero_counts = sum(counts == 0),
                mean_counts = mean(counts,na.rm = TRUE),
                max_counts = max(counts,na.rm = TRUE),
                min_counts = min(counts,na.rm = TRUE),
                variance_counts = var(counts,na.rm = TRUE))

            summary_data$percentage_zero = summary_data$amount_zero_counts / summary_data$amount_genes
            summary_data$amount_non_zero_counts = summary_data$amount_genes - summary_data$amount_zero_counts


            summary_data$sample = k
            summary_data$cell_type = overview_data$data[i]
            summary_data$dataset = overview_data$name[i]

            sample_cell_statistic = rbind(sample_cell_statistic, summary_data)
        
            #write.csv(sample_summary, paste0(output_path_sc, 'D1_Sample_summary', overview_data$name[i], overview_data$data[i], '_', choose_assay, '.csv'))
            if(gene_selection == 'coeqtl'){
            write.csv(sample_gene_statistic, paste0(output_path_sc, 'D1_Sample_gene_statistic_binom', overview_data$name[i], overview_data$data[i], '_', choose_assay, '.csv'))
                }
            if(gene_selection == 'random'){
            write.csv(sample_gene_statistic, paste0(output_path_sc, 'D1_Sample_gene_statistic_binom', overview_data$name[i], overview_data$data[i], '_', choose_assay, '_', gene_selection, '.csv'))  
                }
            #write.csv(sample_cell_statistic, paste0(output_path_sc, 'D1_Sample_cell_statistic', overview_data$name[i], overview_data$data[i], '_', choose_assay, '.csv'))
    
                
            flush.console()
            print(k)
            flush.console()

        
        
        }
        flush.console()
        print(i)
        flush.console()
    
    
        sample_summary = data.frame()
        sample_gene_statistic = data.frame()
        sample_cell_statistic = data.frame()
    }
        
    

Warning message:
"Layer counts isn't present in the assay object; returning NULL"
Warning message:
"There were 4 warnings in `summarise()`.
The first warning was:
ℹ In argument: `bin_size = as.numeric(try(fitdistr(counts, "negative
  binomial")$estimate[1]))`.
ℹ In group 28: `gene = CCL4`.
Caused by warning in `densfun()`:
! NaNs produced
ℹ Run `dplyr::last_dplyr_warnings()` to see the 3 remaining warnings."


[1] "LL007565"


Warning message:
"There were 66 warnings in `summarise()`.
The first warning was:
ℹ In argument: `bin_size = as.numeric(try(fitdistr(counts, "negative
  binomial")$estimate[1]))`.
ℹ In group 40: `gene = YWHAB`.
Caused by warning in `densfun()`:
! NaNs produced
ℹ Run `dplyr::last_dplyr_warnings()` to see the 65 remaining warnings."


[1] "LL007985"


Warning message:
"There were 12 warnings in `summarise()`.
The first warning was:
ℹ In argument: `bin_size = as.numeric(try(fitdistr(counts, "negative
  binomial")$estimate[1]))`.
ℹ In group 83: `gene = CCL5`.
Caused by warning in `densfun()`:
! NaNs produced
ℹ Run `dplyr::last_dplyr_warnings()` to see the 11 remaining warnings."


...

[1] "LL008702"


Warning message:
"There were 102 warnings in `summarise()`.
The first warning was:
ℹ In argument: `bin_size = as.numeric(try(fitdistr(counts, "negative
  binomial")$estimate[1]))`.
ℹ In group 7: `gene = SH2D2A`.
Caused by warning in `densfun()`:
! NaNs produced
ℹ Run `dplyr::last_dplyr_warnings()` to see the 101 remaining warnings."


[1] "LL008389"


Warning message:
"There were 10 warnings in `summarise()`.
The first warning was:
ℹ In argument: `bin_size = as.numeric(try(fitdistr(counts, "negative
  binomial")$estimate[1]))`.
ℹ In group 64: `gene = CALM2`.
Caused by warning in `densfun()`:
! NaNs produced
ℹ Run `dplyr::last_dplyr_warnings()` to see the 9 remaining warnings."


[1] "LL012757"
[1] 1
